In [1]:
import pandas as pd
import numpy as np

In [2]:
import nltk
from nltk.tokenize import word_tokenize

In [3]:
data=pd.read_excel('/content/LabeledText.xlsx')

In [4]:
data.head()

,File Name,Caption,LABEL
0,1.txt,How I feel today #legday #jelly #aching #gym,negative
1,10.txt,@ArrivaTW absolute disgrace two carriages from...,negative
2,100.txt,This is my Valentine's from 1 of my nephews. I...,positive
3,1000.txt,betterfeelingfilms: RT via Instagram: First da...,neutral
4,1001.txt,Zoe's first love #Rattled @JohnnyHarper15,positive


In [5]:
data=data.drop(['File Name'],axis=1)

In [6]:
data.head()

,Caption,LABEL
0,How I feel today #legday #jelly #aching #gym,negative
1,@ArrivaTW absolute disgrace two carriages from...,negative
2,This is my Valentine's from 1 of my nephews. I...,positive
3,betterfeelingfilms: RT via Instagram: First da...,neutral
4,Zoe's first love #Rattled @JohnnyHarper15,positive


In [7]:
("len of dataset",len(data))

('len of dataset', 4869)

In [8]:
("different label",data['LABEL'].unique())

('different label', array(['negative', 'positive', 'neutral'], dtype=object))

In [9]:
data['LABEL'].count()

4869

In [10]:
data['LABEL'].value_counts()

neutral     1771
positive    1646
negative    1452
Name: LABEL, dtype: int64

**data preprocessing**


In [11]:
type(data['Caption'])

pandas.core.series.Series

In [12]:
data['Caption'].head()

0        How I feel today #legday #jelly #aching #gym 
1    @ArrivaTW absolute disgrace two carriages from...
2    This is my Valentine's from 1 of my nephews. I...
3    betterfeelingfilms: RT via Instagram: First da...
4           Zoe's first love #Rattled @JohnnyHarper15 
Name: Caption, dtype: object

In [13]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [14]:
len(data['Caption'])

4869

In [15]:
data['Caption'][217]

'Meet Sir Button available #Compassionate #Adoption #Montreal https://t.co/OAcT2mj6du '

In [16]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [17]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from nltk.stem import PorterStemmer

# Define your preprocessing function
def preprocess(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)

    # Tokenization
    tokens = word_tokenize(text)

    # Convert to lowercase
    tokens = [token.lower() for token in tokens]

    # Remove punctuation
    tokens = [token for token in tokens if token not in string.punctuation]

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    #stemmer
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(token) for token in tokens]

    rev=' '.join(stemmed_tokens)
    return rev



# Apply preprocessing function to the column
data["processed_sentences"] = data['Caption'].apply(preprocess)

# View processed data
print(data["processed_sentences"])


0                         feel today legday jelli ach gym
1       arrivatw absolut disgrac two carriag bangor ha...
2       valentin 's 1 nephew elat sometim littl thing ...
3       betterfeelingfilm rt via instagram first day f...
4                  zoe 's first love rattl johnnyharper15
                              ...                        
4864      omg well done eskom 'man die loadshed powerless
4865                       feelin love valentinesday care
4866                               blue eye ca n't beaten
4867                   la chucha louuu te chupo lo ojo ..
4868    colorsplash_bw zealou remedio herbal tratamien...
Name: processed_sentences, Length: 4869, dtype: object


In [18]:
data['processed_sentences'][0]

'feel today legday jelli ach gym'

In [19]:
data['processed_sentences'][217]

'meet sir button avail compassion adopt montreal'

In [20]:
import pickle
with open('preprocess_model.pkl', 'wb') as preprocess_file:
    pickle.dump(preprocess, preprocess_file)

In [21]:
len(data['processed_sentences'])

4869

In [22]:
corpus=[]
for i in range(0,4869):
  corpus.append(data['processed_sentences'][i])

In [ ]:
# from nltk.corpus import stopwords
# from nltk.stem.porter import PorterStemmer
# ps = PorterStemmer()
# all_stopwords = stopwords.words('english')

In [ ]:
# import re
# corpus=[]
# for i in range(0,4869):
#   rev=re.sub(r'[^a-zA-Z\s]|https?://\S+|www\.\S+','',data['Caption'][i])              # s is for white space
#   rev = rev.lower()
#   rev = rev.split()
#   rev = [ps.stem(word) for word in rev if not word in set(all_stopwords)]
#   rev = ' '.join(rev)
#   corpus.append(rev)


In [23]:
corpus[217]

'meet sir button avail compassion adopt montreal'

In [24]:
from sklearn.preprocessing import LabelEncoder

In [25]:
data['LABEL'].value_counts()

neutral     1771
positive    1646
negative    1452
Name: LABEL, dtype: int64

In [26]:
le=LabelEncoder()
data['LABEL'].unique()
data['LABEL']=le.fit_transform(data['LABEL'])
data['LABEL'].unique()

array([0, 2, 1])

In [27]:
data['LABEL'].value_counts()        # 1- neutral,2-positive,0-negative

1    1771
2    1646
0    1452
Name: LABEL, dtype: int64

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(ngram_range=(1,2),max_features=10000)

In [29]:
x=tfidf.fit_transform(corpus).toarray()
y=data['LABEL']

In [30]:
#creating pickle of vectorizer separately and pickle file of model separately,now creating for vectorizer
import pickle
with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf,f)

In [31]:
from sklearn.naive_bayes import MultinomialNB
nb=MultinomialNB()
param_grid={
    'alpha':[1e-8, 1e-7, 1e-6, 1e-5, 1e-4]
}


In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size = 0.25, random_state = 0)

In [33]:
from sklearn.model_selection import GridSearchCV
gs=GridSearchCV(estimator=nb,param_grid=param_grid,cv=5)
gs.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=MultinomialNB(),
             param_grid={'alpha': [1e-08, 1e-07, 1e-06, 1e-05, 0.0001]})

In [34]:
best_model = gs.best_estimator_

In [35]:
test_score=best_model.score(X_test,y_test)

In [36]:
test_score

0.6018062397372742

In [37]:
y_pred=best_model.predict(X_test)

**random forest calssifier**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
param_grid={
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
gs2=GridSearchCV(estimator=rf, param_grid=param_grid, cv=5)
gs2.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [None, 10, 20],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]})

In [ ]:
print("Best hyperparameters:", gs2.best_params_)
print("Best score:", gs2.best_score_)

Best hyperparameters: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 300}
Best score: 0.694887843636977


**Logistic Regression**


In [42]:
import warnings
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
warnings.filterwarnings("ignore")
lr = LogisticRegression(max_iter=100)
param_grid = {
    'C': [0.01,0.1, 1, 10,100],
    'penalty': ['l2','l1'],
    'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag']
}
grid_search = GridSearchCV(estimator=lr, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.01, 0.1, 1, 10, 100], 'penalty': ['l2', 'l1'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag']})

In [43]:
print("Best hyperparameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

Best hyperparameters: {'C': 10, 'penalty': 'l1', 'solver': 'liblinear'}
Best score: 0.7047444858797295


In [44]:
best_model = grid_search.best_estimator_
test_score = best_model.score(X_test, y_test)

In [45]:
test_score

0.6995073891625616

In [46]:
#creating pickle file of model
import pickle
with open('Sentiment_Model_kaggle_logistic_regression.pkl', 'wb') as f:
    pickle.dump(best_model,f)

In [47]:
import pickle
with open('Sentiment_Model_kaggle_logistic_regression.pkl', 'rb') as f:
    lr = pickle.load(f)

In [48]:
from nltk.corpus import stopwords
all_stopwords = stopwords.words('english')

In [55]:
from nltk.stem import PorterStemmer
new_sentence='fucker guy'
ps=PorterStemmer()
rev_new=re.sub(r'[^a-zA-Z\s]|https?://\S+|www\.\S+','',new_sentence)              # s is for white space
rev_new = rev_new.lower()
rev_new = rev_new.split()
rev_new = [ps.stem(word) for word in rev_new if not word in set(all_stopwords)]
rev_new = ' '.join(rev_new)
print(rev_new)

fucker guy


In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer
with open('tfidf_vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)


In [57]:
vectorized_sentence=vectorizer.transform([rev_new]).toarray()

In [58]:
pred_sent=lr.predict(vectorized_sentence)

In [59]:
pred_sent

array([1])

In [60]:
# gives the coressponding value of encoded label
predicted_word = le.inverse_transform([pred_sent])
predicted_word

array(['neutral'], dtype=object)

**SVC**

In [ ]:
from sklearn.svm import SVC


In [ ]:
svm_model = SVC(kernel='linear', C=1.0)
svm_model.fit(X_train, y_train)

SVC(kernel='linear')

In [ ]:
y_pred2=svm_model.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
accuracy_score(y_test, y_pred2)

0.6798029556650246

In [ ]:
import joblib
print("Joblib version:", joblib.__version__)

Joblib version: 1.3.1
